In [ ]:
import duckdb
import time

# Oppretter DuckDB-tilkobling
con = duckdb.connect()

# Leser fra parquet-filen
query = """
    SELECT *
    FROM read_parquet('hais_2024-12-01.snappy.parquet.geo.parquet')
"""

# Definer batch-størrelse
batch_size = 10000
offset = 0

while True:
    # Bygg spørring for neste del av data
    batch_query = f"{query} LIMIT {batch_size} OFFSET {offset}"
    batch = con.execute(batch_query).fetchdf()
    
    # Sjekker om den har nådd slutten
    if len(batch) == 0:
        print("Ingen flere data. Strømming avsluttes.")
        break
    
    # Viser info om batchen for å vise at data “kommer inn”
    print(f"\nStreamer batch fra offset {offset}:")
    # viserkun de første 5 kolonnene
    print(batch.iloc[:, :5].head())
    
    # For å simulere datastrømming venter den i 2 sek
    time.sleep(2)
    
    # Neste offset
    offset += batch_size

# Lukk tilkoblingen
con.close()